## Step2: Specialization Similarity Ranking

In [4]:
import pickle
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
authid_12 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/2021/A1.2_who_guideline_development_group.npy')
authid_13 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/A1.3_who_systematic_review_team_revise.npy')
authid_14 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/2021/A1.4_who_external_methodologists.npy')
authid_15 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/A1.5_who_external_review_team_revise.npy')

boradmember_authid = list(authid_12)+list(authid_13)+list(authid_14)+list(authid_15)
boradmember_authid1 = set(boradmember_authid)

In [6]:
len(boradmember_authid)

113

In [7]:
authid_list = boradmember_authid

In [8]:
authid_list1 = [35780677200 if x == 7006806977 else 57206422848 if x == 54791745100  else 57219144066 if x == 55588756600  else 57219144066 if x == 57191727610  else 57219144066 if x == 14037485900 else x for x in authid_list]
treatment_authids = list(set(authid_list1))

In [9]:
len(treatment_authids)

108

In [10]:
average_researcher_vectors = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/researcher_average_embedding_matrix_2011_15.npy')

In [11]:
file_path = f'/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/sparse_one_hot_authids_idx.pkl'
with open(file_path, 'rb') as f:
    authid_to_idx = pickle.load(f)

In [13]:
import numpy as np
from scipy.spatial.distance import cdist


# Get the author indices of the treatment group
treatment_indices = np.array([authid_to_idx[aid] for aid in treatment_authids if aid in authid_to_idx.keys()])

# Get the specialization vectors of the treatment group
treatment_vectors = average_researcher_vectors[treatment_indices]  # (108, 1536)

# Get the author indices of the researchers except the treatment group
non_treatment_authids = set(authid_to_idx.keys()) - set(treatment_authids)
non_treatment_authids = set(authid_to_idx.keys())
non_treatment_indices = np.array([authid_to_idx[aid] for aid in non_treatment_authids])

# Get the specialization vectors of the researchers except the treatment group
non_treatment_vectors = average_researcher_vectors[non_treatment_indices]  # (num_researchers - 108, 1536)

# Calculate COS similarities
cosine_similarities = cosine_similarity(treatment_vectors, non_treatment_vectors)  # (108, num_researchers - 108)

# Top 40 most similar researchers with each researcher
top_k = 40 # top_k includes 
top_k_indices = np.argsort(cosine_similarities, axis=1)[:, :top_k:-1]
sorted_similarities = np.sort(cosine_similarities, axis=1)[:, :top_k:-1]

In [14]:
import numpy as np
from scipy.spatial.distance import cdist


# Get the author indices of the treatment group
treatment_indices = np.array([authid_to_idx[aid] for aid in treatment_authids if aid in authid_to_idx.keys()])

# Get the specialization vectors of the treatment group
treatment_vectors = average_researcher_vectors[treatment_indices]  # (108, 1536)

In [15]:
treatment_auhids = np.array([aid for aid in treatment_authids if aid in authid_to_idx.keys()])

treatment_indices_new_idx_to_authid = {}
for index, aid in enumerate(treatment_auhids):
    treatment_indices_new_idx_to_authid[index] = authid_to_idx[aid]

In [16]:
# Calculate COS similarities
cosine_similarities = cosine_similarity(treatment_vectors, average_researcher_vectors)  # (108, num_researchers)

In [17]:
top_indices = np.argsort(-cosine_similarities, axis=1)
sorted_similarities = np.sort(-cosine_similarities, axis=1)

In [18]:
top_k_indice = top_indices[:, 1:top_k+1]

In [19]:
sorted_k_similarities = sorted_similarities[:, 1:top_k+1]

In [20]:
from collections import defaultdict

def invert_dict(d):
    inverted = defaultdict(list)
    for k, v in d.items():
        inverted[v].append(k)
    return inverted

idx_to_authid = invert_dict(authid_to_idx)

In [21]:
top_k_authids = []
for indice in top_k_indice:
    tmp = []
    for index in indice:
        tmp_num = idx_to_authid[index][0]
        tmp.append(tmp_num)
    top_k_authids.append(tmp)

In [22]:
top_k_cossim_authid_dict = {}
for index,top_authids in enumerate(top_k_authids):
    top_k_cossim_authid_dict[treatment_auhids[index]] = top_authids
    

In [99]:
# save the result
# np.save("/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/top_k_similar_authids.npy", top_k_cossim_authid_dict)